# Cargando datos con API-REST

## ¿Qué es API-REST?

**API** es la **A**pplication **P**rogramming **I**nterface, es decir nuestra forma de interactuar con una aplicación. **REST** quiere decir **RE**presentational **S**tate **T**ransfer. Normalmente cuando hablamos de REST, estamos hablando de una interfaz para obtener datos usando directamente HTTP, es decir es un tipo de web API. 

> **De manera sencilla podemos decir que es una Interfaz con la que se interactúa mediante URLs normalmente para obtener datos.**

La principal diferencia entre esto y una URL común, es que la URL de una página web devolverá algo que tu navegador puede interpretar y mostrar de forma "bonita", mientras que una API web mandará datos o instrucciones útiles para ti o tu ordenador.

In [1]:
from IPython.display import HTML

Veamos la respuesta a una página web clásica

In [2]:
HTML('<iframe src="https://developer.github.com/v3/" width="700" height="400"></iframe>')

Veamos la respuesta de una web api

In [3]:
HTML(url="https://api.github.com/")

En definitiva, cuando utilizamos una WEB API, están involucrados:

* Cliente: realiza la petición a través de la url
* Servidor: da una respuesta a la url recibida
* Protocolo: se establece en la especificación de la API

## ¿Cómo utilizar una API?

El uso de WEB APIs permite: obtener información que sería cosotsa de obtener y procesar de otra manera (incluso en tiempo real).

En algunas ocasiones, la API es pública y cualquiera puede hacer una petición, pero en otras es necesario tener una *api key* que nos identifica. Por lo tanto, el proceso para obtener datos suele ser:

1. Comprobar si existe una API-REST para obtener dichos datos
2. Obtener una api key en caso de que sea necesaria
3. Leer la especificación de la API para saber cómo componer la URL y como interpretar la respuesta
4. Utilizar la API, en nuestro caso desde Python.

### Ejemplo: descargando datos de AEMET 

Esta es la página principal de su API:

[https://opendata.aemet.es/centrodedescargas/inicio](https://opendata.aemet.es/centrodedescargas/inicio)
    
Aquí podemos encontrar: información general, el lugar donde obtener nuestra API key, una interfaz para acceder a los datos para público general 

In [4]:
HTML('<iframe src="https://opendata.aemet.es/centrodedescargas/inicio" width="1000" height="400"></iframe>')

## Módulo requests

Aunque existen otras formas y librería para trabajar con HTTP en Python, el módulo `requests` está específicamente pensado para trabajar con APIs web.

Como siempre hasta ahora, lo primero es importarlo:

In [5]:
import requests

Necesitaremos cargar nuesta API key, lo más cómodo es almacenarla en un fichero y cargarla desde ahí. Creemos una función para leerla:

In [6]:
def load_api_key(file):
    """Returns the contents in the file without the final line break
    """
    with open(file, 'r') as f:
        api_key = f.read().rstrip()
    return api_key

In [7]:
api_key = load_api_key("../../apikey-aemet.txt")

Debemos saber cuál es la url a la que vamos a hacer la petición:

In [8]:
# Predicción costera de Asturias, Cantabria y País Vasco debemos introducir
url = "https://opendata.aemet.es/opendata/api/prediccion/maritima/costera/costa/41"

querystring = {"api_key": api_key}

Por último, lanzamos la petición:

In [9]:
response = requests.get(url, params=querystring, verify=False)

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Comprobando la respuesta:

In [10]:
response

<Response [200]>

Vemos que hemos obtenido una respuesta con código 200, esto quiere decir, en el código habitual de las API-REST, que todo ha ido bien. De hecho, es conveniente ver que todo ha ido bien antes de hacer nada más:

In [11]:
response.status_code == requests.codes.ok

True

Otra información que podemos obtener es, por ejemplo;

In [12]:
# Tiempo en procesar la petición
print("Elapsed: ", response.elapsed)
# Información del servidor
print("Headers: ", response.headers)

Elapsed:  0:00:00.087180
Headers:  {'Server': 'Apache/2.2.15 (CentOS)', 'Date': 'Tue, 30 May 2017 23:10:29 GMT'}


Pero... ¿dónde están nuestros datos?

In [13]:
response.content

b'{\n  "descripcion" : "exito",\n  "estado" : 200,\n  "datos" : "https://opendata.aemet.es/opendata/sh/43cdfb13",\n  "metadatos" : "https://opendata.aemet.es/opendata/sh/8846af34"\n}'

¡Parece que esto es un json!

In [14]:
content = response.json()

content

{'datos': 'https://opendata.aemet.es/opendata/sh/43cdfb13',
 'descripcion': 'exito',
 'estado': 200,
 'metadatos': 'https://opendata.aemet.es/opendata/sh/8846af34'}

Efectivamente, la mayoría de las WEB APIs devuelven json o xml. Pero, otra vez... ¿dónde están nuestros datos?

In [15]:
r_meta = requests.get(content['metadatos'], verify=False)
r_data = requests.get(content['datos'], verify=False)

if r_meta.status_code == requests.codes.ok:
    metadata = r_meta.json()
    
if r_data.status_code == requests.codes.ok:
    data = r_data.json()

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url'

In [16]:
print(metadata[0].keys())
print(data[0].keys())

dict_keys(['campos', 'descripcion', 'unidad_generadora', 'formato', 'periodicidad'])
dict_keys(['aviso', 'situacion', 'prediccion', 'tendencia', 'nombre', 'origen', 'id'])


In [17]:
metadata[0]['campos']

[{'origen': [{'descripcion': 'Agencia Estatal de Meteorología - AEMET. Gobierno de España',
    'id': 'productor',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': 'URL de la Web de AEMET - http://www.aemet.es',
    'id': 'web',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': 'es',
    'id': 'language',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
    'id': 'copyright',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': 'http://www.aemet.es/es/nota_legal/nota_legal',
    'id': 'notaLegal',
    'requerido': True,
    'tipo_datos': 'anyURI'},
   {'descripcion': 'Fecha de elaboración',
    'id': 'elaborado',
    'requerido': True,
    'tipo_datos': 'dataTime'},
   {'descripcion': 'Fecha de inicio de la validez de los avisos',
    'id': 'inicio',
    'requerido': True,
    'tipo_datos': 'dataTim

In [18]:
data[0]['prediccion']

{'fin': '2017-05-31',
 'inicio': '2017-05-30',
 'zona': [{'id': 8033010,
   'nombre': 'Aguas costeras de Asturias',
   'subzona': {'id': 8033010,
    'nombre': 'Aguas costeras de Asturias',
    'texto': 'PRINCIPALMENTE ESTE Y NORDESTE FUERZA 3 O 4. MAREJADILLA O MAREJADA. TEMPORALMENTE REGULAR O LOCALMENTE MALA. BANCOS DE NIEBLA.'}},
  {'id': 8063910,
   'nombre': 'Aguas costeras de Cantabria',
   'subzona': {'id': 8063910,
    'nombre': 'Aguas costeras de Cantabria',
    'texto': 'ESTE Y NORDESTE FUERZA 3 O 4. MAREJADILLA O MAREJADA.'}},
  {'id': 8154810,
   'nombre': 'Aguas costeras de Bizkaia',
   'subzona': {'id': 8154810,
    'nombre': 'Aguas costeras de Bizkaia',
    'texto': 'VARIABLE, PRINCIPALMENTE NORTE Y NORDESTE, FUERZA 1 A 4. MAREJADILLA O RIZADA.'}},
  {'id': 8152010,
   'nombre': 'Aguas costeras de Gipuzkoa',
   'subzona': {'id': 8152010,
    'nombre': 'Aguas costeras de Gipuzkoa',
    'texto': 'VARIABLE, PRINCIPALMENTE NORTE Y NORDESTE, FUERZA 1 A 4. MAREJADILLA O RIZAD

En definitiva, podríamos reagrupar todo lo anterior como:

In [19]:
from warnings import warn

def get_prediction_for_cantabria(api_key):
    url = "https://opendata.aemet.es/opendata/api/prediccion/maritima/costera/costa/41"
    querystring = {"api_key": api_key}
    
    response = requests.get(url, params=querystring, verify=False)
    
    prediction = None
     
    if response.status_code == requests.codes.ok:
        r_data = requests.get(content['datos'], verify=False)
        if r_data.status_code == requests.codes.ok:
            data = r_data.json()
            prediction = data[0]['prediccion']
    
    elif response.status_code == requests.codes.TOO_MANY_REQUESTS:
        warn('Too many requests')
        
    elif response.status.code == requests.codes.NOT_FOUND:
        warn('No data for the response')
        
    else:
        warn('Code error {}'.format(response.status_code))
            
    return prediction
        

In [21]:
# Raising too many requests warining
for ii in range(50):
    print(get_prediction_for_cantabria(api_key))

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'inicio': '2017-05-30', 'zona': [{'subzona': {'nombre': 'Aguas costeras de Asturias', 'texto': 'PRINCIPALMENTE ESTE Y NORDESTE FUERZA 3 O 4. MAREJADILLA O MAREJADA. TEMPORALMENTE REGULAR O LOCALMENTE MALA. BANCOS DE NIEBLA.', 'id': 8033010}, 'nombre': 'Aguas costeras de Asturias', 'id': 8033010}, {'subzona': {'nombre': 'Aguas costeras de Cantabria', 'texto': 'ESTE Y NORDESTE FUERZA 3 O 4. MAREJADILLA O MAREJADA.', 'id': 8063910}, 'nombre': 'Aguas costeras de Cantabria', 'id': 8063910}, {'subzona': {'nombre': 'Aguas costeras de Bizkaia', 'texto': 'VARIABLE, PRINCIPALMENTE NORTE Y NORDESTE, FUERZA 1 A 4. MAREJADILLA O RIZADA.', 'id': 8154810}, 'nombre': 'Aguas costeras de Bizkaia', 'id': 8154810}, {'subzona': {'nombre': 'Aguas costeras de Gipuzkoa', 'texto': 'VARIABLE, PRINCIPALMENTE NORTE Y NORDESTE, FUERZA 1 A 4. MAREJADILLA O RIZADA.', 'id': 8152010}, 'nombre': 'Aguas costeras de Gipuzkoa', 'id': 8152010}], 'fin': '2017-05-31'}


/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3

{'inicio': '2017-05-30', 'zona': [{'subzona': {'nombre': 'Aguas costeras de Asturias', 'texto': 'PRINCIPALMENTE ESTE Y NORDESTE FUERZA 3 O 4. MAREJADILLA O MAREJADA. TEMPORALMENTE REGULAR O LOCALMENTE MALA. BANCOS DE NIEBLA.', 'id': 8033010}, 'nombre': 'Aguas costeras de Asturias', 'id': 8033010}, {'subzona': {'nombre': 'Aguas costeras de Cantabria', 'texto': 'ESTE Y NORDESTE FUERZA 3 O 4. MAREJADILLA O MAREJADA.', 'id': 8063910}, 'nombre': 'Aguas costeras de Cantabria', 'id': 8063910}, {'subzona': {'nombre': 'Aguas costeras de Bizkaia', 'texto': 'VARIABLE, PRINCIPALMENTE NORTE Y NORDESTE, FUERZA 1 A 4. MAREJADILLA O RIZADA.', 'id': 8154810}, 'nombre': 'Aguas costeras de Bizkaia', 'id': 8154810}, {'subzona': {'nombre': 'Aguas costeras de Gipuzkoa', 'texto': 'VARIABLE, PRINCIPALMENTE NORTE Y NORDESTE, FUERZA 1 A 4. MAREJADILLA O RIZADA.', 'id': 8152010}, 'nombre': 'Aguas costeras de Gipuzkoa', 'id': 8152010}], 'fin': '2017-05-31'}
{'inicio': '2017-05-30', 'zona': [{'subzona': {'nombre':

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3

None
None
None
None


/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3

None
None


/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
